# PROGRES - Mini-Project 3
# DHT Chord 

Fabien Mathieu - fabien.mathieu@normalesup.org

Sébastien Tixeuil - Sebastien.Tixeuil@lip6.fr

The goal of this mini-project is to build a DHT simulator to test its performance:
Table construction, distributed content management, load balancing and traffic analysis.

The table will be used to manage Wikipedia pages. **Exceptionally**, the peer in charge of a page will also be responsible for retrieving and making available the content of the page. Recall that this is not the typical behavior of a DHT, which is only supposed to point to the peers owning the content. This overload is introduced here in order to reduce the implementation complexity of the mini-project.

# Exercise 1. Creation of a Peer class

Realize in Python a `Peer` class allowing to simulate a DHT Chord.

The communication will be simulated through a dictionary `dht`: `hash` -> `Peer`: knowing the hash of a peer allows to contact it (in real life, in addition to the hash the IP address is known), and the network calls within the DHT will be represented by calls to the key of the destination in the dictionary.

The hash function to use is SHA1, available from the `hashlib` library https://docs.python.org/3/library/hashlib.html

SHA1 provides 160-bit hashes.

Instances of the `Peer` class will have to implement (at least) the following attributes / methods:
- `id` : peer identifier (integer)
- `hash` : the hash of the peer identifier
- `predecessor` : the hash of the peer's predecessor
- `successor` : the hash of the successor of the peer
- `finger`: the finger table of the peer (see course)
- `update_finger(hash_list)`: a function that uses a list of peer hashes to update the finger table, i.e. add missing shortcuts or replace existing shortcuts with better ones.
- `give_neighbors()`: a function that returns a list of known hashes (its own hash, the hash of the predecessor, the hash of the successor, the hashes of the `finger`)
- `refresh_finger()`: A function that refreshes the finger table by asking each neighbor for its list of neighbors
- `isincharge(key)`: a function that indicates whether the peer is in charge of `key
- `lookup(key, dht)`: a function that returns the hash of the peer in charge of `key` as well as the distance, in number of hops, that separates the peer performing the *lookup* from the peer in charge
- `table`: see below 
- `publish_page(page_title, dht)`: see below
- `retrieve_page(page_title)`: see below
- `search(search_string, dht)`: see below

The `table` attribute contains the dictionary of keys managed by the peer; the table may contain two types of keys:
  - word keys: to a given word we associate the hash of the word prefixed by `word:` (e.g. `computer` is represented by the SHA1 hash of `word:computer`). The associated value in the table is a set (`set` Python), which will typically contain the titles of wikipedia pages that contain the word.
  - page keys: to a given wikipedia page we associate the hash of the page title prefixed by `page:` (for example `Circuit (computer science)` is represented by the SHA1 hash of `page:Circuit (computer science)`). The associated value in the table is the content of the page.

The `publish_page` method should perform the following actions:
- Separate the title into words written in lower case. For example, `Circuit (computer science)` becomes `["circuit", "computer", "science"]`, `Object-oriented programming` becomes `["object", "oriented", "programming"]`.
- For each word, find the peer responsible for the word key. If this key exists, add the page title to the associated set, otherwise initialize the key (with the value of a one-element set, the page title). Remember that the update is done by the responsible peer, not by the peer calling the method.




The `publish_page` method is not in charge of retrieving the content of the page itself.

The `retrieve_page` method is responsible for providing the content. It should perform the following actions:
- If the peer is responsible for the page, return the content:
  - If the key is present in the table, return the associated value
  - If the key is not present, use the wikipedia module to retrieve the page from the title (method `wikipedia.page`), store the content (attribute `content` of the result) in the table and return it
  - In case of an error from the wikipedia module (this can happen, but quite rarely), store nothing and return ``Page not found``.
- If the peer is not responsible for the page, retrieve the content from the responsible peer.

The `search` method should: 
- Separate the search into words written in lower case.
- Retrieve for each word the set of Wikipedia pages containing the word (if the word is not indexed, it is the empty set)
- Return the list of titles containing all the words of the search.
- If the list is empty but at least one word is indexed, issue a warning (see https://docs.python.org/3/library/logging.html) and return the union (titles containing at least one of the words of the search).
- If no word is indexed, issue a warning that none of the words were found and return an empty list.

The initialization of a peer will take three arguments:
- The identifier (integer)
- The DHT (the `dht` dictionary)
- The entry point identifier

If the DHT is empty, the peer is simply added to it (it is its own predecessor and successor, and its finger is empty). Otherwise, it is inserted via the entry point, as seen in the course and in TME.

**Note**: `table`, `publish_page`, `retrieve_page` and `search` will only be used from exercise 3 on. You can code a first version that does not contain these attributes/methods, and update at the time of exercise 3.

# Exercise 2. DHT creation

From an empty `dht` dictionary, insert 10000 pairs of identifier 0 to 9999 in this order, using 0 as the insertion point.
- What is the average size of the finger tables? The size of the smallest finger? The size of the largest finger?
- From the peer with identifier 42, identify with the `lookup` method the distance (in hops) that separates it from the successors of each of the 10000 peers. What is the average distance? The greatest distance?
- Have each peer run the `refresh_finger` method and repeat the two previous questions. What do you notice?
- Repeat the previous question. What do you notice?

# Exercise 3. Publishing pages from Wikipedia

Using the wikipedia module and the `links` method of the `WikipediaPage` objects, retrieve the set of pages located at most 2 hops away from the `Computer science` page, i.e. :
- `Computer science`
- The titles of the pages referenced by the `Computer science` page
- The titles of the pages referenced by the pages referenced by the `Computer science` page
- After eliminating duplicates, save the list to a file (so you can re-use it later) and publish all the titles in the DHT. You can use an arbitrary peer to call the method.

- How many words are indexed in total?
- What is the average size of the peer tables? The size of the largest table?

# Exercise 4. Using the DHT

- Start by manually testing how the DHT works: from any peer (e.g. ID 42), perform some searches (`search`) and retrieve some content (`retrieve_page`). Do not hesitate to test partial searches (empty intersection) and unsuccessful searches.
- Programmatically, retrieve from a peer the content of 2000 pages among those published (you can use the list of titles you have saved). What is the largest number of articles indexed by a peer?